# Install requirements for selenium

In [1]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.4 MB 3.7 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.4 MB 3.7 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.4 MB 3.6 MB/s eta 0:00:02
   --------------- ------------------------ 3.7/9.4 MB 3.7 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.4 MB 3.6 MB/s eta 0:00:02
   ---------------------- ----------------- 5.2/9.4 MB 3.7 MB/s eta 0:00:02
   ------------------------ --------------- 5.8/9.4 MB 3.7 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.4 MB 3.6 MB/s eta 0:00:01
   ------------------------------- -------- 7.3/9.4 MB 3.7 MB/s eta 0:00:01
   ---------------------------------- ----- 8.1/9.4 MB 3.6 MB/s eta 0:00:01
   ------------------------------------- -- 8.9/9.4 MB 3.6 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


# Import modules

In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
import json

# Request page source from URL

In [3]:
# Identitas browser saat akses halaman web karena IMDb menolak (403 Forbidden) jika tidak ada user-agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"

In [4]:
# Keperluan untuk penggunaan selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--diable-dve-shm-uage')
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=options)

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [5]:
driver.get('https://www.imdb.com/list/ls574761151/')

In [6]:
time.sleep(5)  # tunggu JavaScript selesai load

# Ambil page source dan parse dengan BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Print seluruh HTML dengan struktur yang rapi
print(soup.prettify())

<html class="scriptsOn" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <script async="" src="https://sb.scorecardresearch.com/beacon.js">
  </script>
  <script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/216YVwoRFDL.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <title>
   Top 100 Anime TV Series (IMDB Ratings)
  </title>
  <meta content="Top 100 Anime TV Series as rated by IMDb Users


* Shorts anime TV series and documentaries are not included
* To be included on the list, a movie must receive at least 5.000 ratings
* Movie must receive ratings from at least 7.5 score


Contact: facebook.com/thanhthiftu2" data-id="main" name="description"/>
  <meta content="0cadf7898134e79b" name="google-site-verification"/>
  <meta content="C1DAC

# Scraped the data

In [7]:
# Dapatkan konten html yang secara khusus memakai tag li dan class yang bersesuaian (lokasi dimana informasi data yang ingin dicari, dilihat dari inspect)
scraped_animes = soup.find_all('li', class_='ipc-metadata-list-summary-item')
scraped_animes

[<li class="ipc-metadata-list-summary-item"><div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t ipc-btn--not-interactable"></span><div class="sc-52ea7f05-1 ejhgHs dli-parent"><div class="sc-52ea7f05-0 YKXtw"><div class="sc-d0224b4e-0 jfogmY dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--wl-true ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--media-radius ipc-media--base ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%"><img alt="Kappei Yamaguchi, Shidô Nakamura, and Mamoru Miyano in Death Note (2006)" class="ipc-image" loading="lazy" sizes="50vw, (min-width: 480px) 34vw, (min-width: 600px) 26vw, (min-width: 1024px) 16vw, (min-width: 1280px) 16vw" src="https://m.media-a

In [8]:
# Cek data yang didapat
all_target_li = soup.find_all('li', class_='ipc-metadata-list-summary-item')
print(f"Total yang cocok: {len(all_target_li)}")

Total yang cocok: 99


In [12]:
# parse anime
# Note: digunakan find untuk dapatkan elemen tunggal, jika tidak maka digunakan select
anime_list = []
star_dict = {}
star_id_counter = 1

for li in scraped_animes:
    try:
        # Judul
        title_tag = li.find("h3", class_="ipc-title__text")
        title = title_tag.get_text(strip=True) if title_tag else None

        # Metadata: tahun, episode, rating age, title type
        metadata_spans = li.select(".dli-title-metadata span")
        title_type_span = li.select_one(".dli-title-type-data")

        year = metadata_spans[0].get_text(strip=True) if len(metadata_spans) > 0 else None
        episodes = metadata_spans[1].get_text(strip=True) if len(metadata_spans) > 1 else None
        age_rating = metadata_spans[2].get_text(strip=True) if len(metadata_spans) > 2 else None
        title_type = title_type_span.get_text(strip=True) if title_type_span else None

        # Rating IMDb
        rating_tag = li.find("span", class_="ipc-rating-star--rating")
        rating = rating_tag.get_text(strip=True) if rating_tag else None

        # Poster image
        img_tag = li.find("img")
        poster_url = img_tag["src"] if img_tag else None

        # Deskripsi pendek
        description_div = li.find("div", class_="title-description-plot-container")
        description = description_div.get_text(strip=True) if description_div else None

        # Stars (Seiyu/Pengisi suara)
        star_span = li.select(".title-description-credit a")
        stars = [a.get_text(strip=True) for a in star_span]

        star_ids = []
        for star_name in stars:
            if star_name not in star_dict:
                star_dict[star_name] = star_id_counter
                star_id_counter += 1
            star_ids.append(star_dict[star_name])

        # Simpan dalam dictionary
        anime = {
            "title": title,
            "year": year,
            "episodes": episodes,
            "age_rating": age_rating,
            "title_type": title_type,
            "rating": rating,
            "poster_url": poster_url,
            "description": description,
            "stars_ids": star_ids
        }

        anime_list.append(anime)

    except Exception as e:
        print("Error saat parsing:", e)


In [14]:
stars_list = [{"id": id, "name": name} for name, id in star_dict.items()]
# Lihat seluruh list seiyu yang didapatkan
for star in stars_list:
    print(star)

{'id': 1, 'name': 'Mamoru Miyano'}
{'id': 2, 'name': 'Brad Swaile'}
{'id': 3, 'name': 'Vincent Tong'}
{'id': 4, 'name': 'Jessie James Grelle'}
{'id': 5, 'name': 'Bryce Papenbrook'}
{'id': 6, 'name': 'Trina Nishimura'}
{'id': 7, 'name': 'Hiromu Arakawa'}
{'id': 8, 'name': 'Kent Williams'}
{'id': 9, 'name': 'Iemasa Kayumi'}
{'id': 10, 'name': 'Vic Mignogna'}
{'id': 11, 'name': 'Yusuke Murata'}
{'id': 12, 'name': 'Makoto Furukawa'}
{'id': 13, 'name': 'Kaito Ishikawa'}
{'id': 14, 'name': 'Max Mittelman'}
{'id': 15, 'name': 'Doc Harris'}
{'id': 16, 'name': 'Christopher Sabat'}
{'id': 17, 'name': 'Scott McNeil'}
{'id': 18, 'name': 'Kôichi Yamadera'}
{'id': 19, 'name': 'Unshô Ishizuka'}
{'id': 20, 'name': 'Megumi Hayashibara'}
{'id': 21, 'name': 'Masashi Kishimoto'}
{'id': 22, 'name': 'Alexandre Crepet'}
{'id': 23, 'name': 'Junko Takeuchi'}
{'id': 24, 'name': 'Maile Flanagan'}
{'id': 25, 'name': 'Eiichirô Oda'}
{'id': 26, 'name': 'Mayumi Tanaka'}
{'id': 27, 'name': 'Akemi Okamura'}
{'id': 28,

In [15]:
# Lihat seluruh list anime yang didapatkan
for anime in anime_list:
    print(anime)

{'title': '1. Death Note', 'year': '2006–2007', 'episodes': '37 eps', 'age_rating': 'TV-14', 'title_type': 'TV Series', 'rating': '8.9', 'poster_url': 'https://m.media-amazon.com/images/M/MV5BYTgyZDhmMTEtZDFhNi00MTc4LTg3NjUtYWJlNGE5Mzk2NzMxXkEyXkFqcGc@._V1_QL75_UX90_CR0,1,90,133_.jpg', 'description': 'An intelligent high school student goes on a secret crusade to eliminate criminals from the world after discovering a notebook capable of killing anyone whose name is written into it.', 'stars_ids': [1, 2, 3]}
{'title': '2. Attack on Titan', 'year': '2013–2023', 'episodes': '98 eps', 'age_rating': 'TV-MA', 'title_type': 'TV Series', 'rating': '9.1', 'poster_url': 'https://m.media-amazon.com/images/M/MV5BZjliODY5MzQtMmViZC00MTZmLWFhMWMtMjMwM2I3OGY1MTRiXkEyXkFqcGc@._V1_QL75_UY133_CR2,0,90,133_.jpg', 'description': 'After his hometown is destroyed, young Eren Jaeger vows to cleanse the earth of the giant humanoid Titans that have brought humanity to the brink of extinction.', 'stars_ids': [4

# Store the Scraped Data

In [16]:
# Simpan ke file json
with open("../data/anime.json", "w", encoding="utf-8") as f:
    json.dump(anime_list, f, ensure_ascii=False, indent=4)

with open("../data/stars.json", "w", encoding="utf-8") as f:
    json.dump(stars_list, f, ensure_ascii=False, indent=4)